In [1]:
from Mediapipe import MediapipeHands
import pandas as pd
import os

initial_dir = os.getcwd()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [2]:
mediapipeHands = MediapipeHands()

In [3]:
initial_sequence_id = mediapipeHands.sequence_id
initial_sequence_id

0

In [4]:
training_dir = f'{initial_dir}/data/asl_videos'
mediapipeHands.extract_coordinates_from_dir(training_dir)

Hay 529 videos a extraer coordenadas


In [ ]:
training_dir = f'{initial_dir}/data/videos_lensegua'
mediapipeHands.extract_coordinates_from_dir(training_dir)

In [ ]:
training_dir = f'{initial_dir}/data/videos_lensegua_2'
mediapipeHands.extract_coordinates_from_dir(training_dir)

In [5]:
validation_dir = f'{initial_dir}/data/asl_val'
mediapipeHands.extract_coordinates_from_dir(validation_dir, is_val=True)

Hay 109 videos a extraer coordenadas


In [ ]:
validation_dir = f'{initial_dir}/data/videos_val_2'
mediapipeHands.extract_coordinates_from_dir(validation_dir, is_val=True)

In [ ]:
validation_dir = f'{initial_dir}/data/videos_val_3'
mediapipeHands.extract_coordinates_from_dir(validation_dir, is_val=True)

In [ ]:
validation_dir = f'{initial_dir}/data/videos_val_4'
mediapipeHands.extract_coordinates_from_dir(validation_dir, is_val=True)

In [6]:
df = mediapipeHands.get_padded_data()
validation_sequence_ids = mediapipeHands.validation_sequence_ids

In [7]:
validation_sequence_ids

[428,
 429,
 430,
 431,
 432,
 433,
 434,
 435,
 436,
 437,
 438,
 439,
 440,
 441,
 442,
 443,
 444,
 445,
 446,
 447,
 448,
 449,
 450,
 451,
 452,
 453,
 454,
 455,
 456,
 457,
 458,
 459,
 460,
 461,
 462,
 463,
 464,
 465,
 466,
 467,
 468,
 469,
 470,
 471,
 472,
 473,
 474,
 475,
 476,
 477,
 478,
 479,
 480,
 481,
 482,
 483,
 484,
 485,
 486,
 487,
 488,
 489,
 490,
 491,
 492,
 493,
 494,
 495,
 496,
 497,
 498,
 499,
 500,
 501,
 502,
 503,
 504,
 505,
 506,
 507,
 508,
 509,
 510,
 511,
 512,
 513]

In [8]:
if (len(validation_sequence_ids) > 0):
    validation_path = f'{initial_dir}/data/asl/validation.csv'
    validation_df = df[df['sequence_id'].isin(validation_sequence_ids)][["sequence_id", "target", "file"]]
    
    train_path = f'{initial_dir}/data/asl/train.csv'
    train_df = df[~df['sequence_id'].isin(validation_sequence_ids)][["sequence_id", "target", "file"]]
    

    if (initial_sequence_id != 0):
        past_data = pd.read_csv(validation_path)
        final_df = pd.concat([past_data, validation_df])
        final_df.to_csv(validation_path, index=False)

        past_data = pd.read_csv(train_path)
        final_df = pd.concat([past_data, train_df])
        final_df.to_csv(train_path, index=False)
    else:
        validation_df.to_csv(validation_path, index=False)
        train_df.to_csv(train_path, index=False)

In [9]:
data_path = f'{initial_dir}/data/asl/data.csv'
df_final = df.drop("file", axis=1)

if (initial_sequence_id != 0):
    past_data = pd.read_csv(data_path)
    final_df = pd.concat([past_data, df_final])
    final_df.to_csv(data_path, index=False)
else:
    df_final.to_csv(data_path, index=False)

In [10]:
mediapipeHands = MediapipeHands()

In [11]:
initial_sequence_id = mediapipeHands.sequence_id
initial_sequence_id

0

In [12]:
extra_val_dir = f'{initial_dir}/data/test videos asl'
mediapipeHands.extract_coordinates_from_dir(extra_val_dir)

df = mediapipeHands.get_padded_data()

Hay 14 videos a extraer coordenadas


In [13]:
data_path = f'{initial_dir}/data/asl/data_tests.csv'

if (initial_sequence_id != 0):
    past_data = pd.read_csv(data_path)
    final_df = pd.concat([past_data, df])
    final_df.to_csv(data_path, index=False)
else:
    df.to_csv(data_path, index=False)